In [26]:
import enchant
from PIL import Image, ImageEnhance
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
import cv2
import re 
import matplotlib.pyplot as plt
import numpy as np
from imgProcessor.measure.sharpness import parameters
import imutils
from scipy.signal import convolve2d
import math
from skimage.transform import rotate
from skimage.util import random_noise

In [27]:
np.random.seed(2)
img_teste_ids = np.random.randint(0,200, (40))
print(img_teste_ids)

[168  15  72  22  43  75 104   7 162 177  95  75  47  63  31 148 124 116
  37 167 195 102   4 170 107  51 103  38  33  58 124  67  69  88 196  46
 198  95 121  31]


In [28]:

def accuracyByLevenshteinDistance(textOCR,label):
    #tira espaços, tab e \n
    text = re.sub(r"[\n\t\s]*", "", textOCR)
    labelText = re.sub(r"[\n\t\s]*", "", label)
    
    #calcula a distancia de levenshtein entre os textos
    levDistance = enchant.utils.levenshtein(text, labelText)
    
    #calcula a acurácia fazendo accuracy = n-erros/n
    accuracy = (len(labelText)-levDistance) / len(labelText)
    return accuracy

def blurLevel(image):
    return cv2.Laplacian(image, cv2.CV_64F).var()


def biggest_contour(contours):
        biggest = np.array([])
        max_area = 0
        for i in contours:
            area = cv2.contourArea(i)
            if area > 58000:
                peri = cv2.arcLength(i, True)
                approx = cv2.approxPolyDP(i, 0.015 * peri, True)
                if area > max_area and len(approx) == 4:
                    biggest = approx
                    max_area = area
        return biggest
 

def order_points(pts):
        rect = np.zeros((4, 2), dtype = "float32")

        s = np.sum(pts,axis = 1)

        rect[0] = pts[np.argmin(s)]
        rect[2] = pts[np.argmax(s)]
        
        diff = np.diff(pts, axis = 1)
        rect[3] = pts[np.argmin(diff)]
        rect[1] = pts[np.argmax(diff)]
     
        # return the ordered coordinates
        return rect
       
def getAngle(img):
    image = imutils.resize(img, height = 500)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    gray = cv2.bilateralFilter(gray, 20, 30, 30)
    edged = cv2.Canny(gray, 50, 150)
    
  
    cnts = None
    cnts, hierarchy  = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:10]

    
    screenCnt = None
    screenCnt = biggest_contour(cnts)
    
    if(len(screenCnt)==0 ):
        edged = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,13,0.5)
        cnts = None
        cnts, hierarchy  = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:10]
        
        screenCnt = None
        screenCnt = biggest_contour(cnts)

    if(len(screenCnt) >0 ):
        cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 1)
        screenCnt = order_points(screenCnt[:,0,:])
        angle = cv2.minAreaRect(screenCnt)[-1]
        angle = 90 - angle if (angle>45) else angle        
        
        return angle
    else:
        return None


def estimate_noise(I):
    
    I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
    Image  = cv2.Sobel(src=I, ddepth=cv2.CV_64F, dx=1, dy=1, ksize=5) #
    
    H, W = Image.shape

    M = [[1, -2, 1],
        [-2, 4, -2],
        [1, -2, 1]]

    sigma = np.sum(np.sum(np.absolute(convolve2d(Image, M))))

    sigma = sigma * (math.sqrt(math.pi/2) * (1 /  (6 * (W-2) * (H-2))))

    return sigma

In [29]:
accuracyAll = np.array([])
accuracyGood = np.array([])
accuracyMedium = np.array([])
accuracyBad = np.array([])

accuracyAllOriginal = np.array([])

indexGood = np.array([])
indexMedium = np.array([])
indexBad = np.array([])



for i in range(0,200):
    if(i in img_teste_ids): continue
    
    img = np.array([])
    img = cv2.imread('../images/1'+str(i).zfill(3) +'-receipt.jpg')
    if (i == 25 or i == 34 or i == 41 or i == 45 or i == 63 or i == 109 or i == 137 or i == 169 or i == 194 or i == 192): continue    
    
    label = open('../images/1'+str(i).zfill(3)+'-receipt.txt', 'r') 
    labelText = label.read()
    
    imgBlurLevel = blurLevel(img)
    noiseLevel = estimate_noise(img)
    imgAngle = getAngle(img)
    
    text =''
    text = pytesseract.image_to_string(img)
    accuracy = accuracyByLevenshteinDistance(text,labelText)
    accuracyAll = np.append(accuracyAll, accuracy)
    accuracyAllOriginal = np.append(accuracyAllOriginal,accuracy)
    
    #boas - aceitaveis
    if((imgBlurLevel <= 1090.24 and imgBlurLevel >= 354.63 ) and (noiseLevel>= 8.81 and noiseLevel <= 15.32 )): #verifica ruido e nitidez
        if(imgAngle != None): # verifica se conseguiu calcular um angulo
            if(imgAngle <= 2.30):# se encaixa em todos os parametros classifica como bom
                indexGood = np.append(indexGood,i)
                accuracyGood = np.append(accuracyGood, accuracy)
        else:
            indexGood = np.append(indexGood,i)
            accuracyGood = np.append(accuracyGood, accuracy)
                        
    
            
            
for i in range(0,200):
    if(i in img_teste_ids): continue    
    img = np.array([])
    img = cv2.imread('../images/1'+str(i).zfill(3) +'-receipt.jpg')
    if (i == 25 or i == 34 or i == 41 or i == 45 or i == 63 or i == 109 or i == 137 or i == 169 or i == 194 or i == 192): continue
    if(len(indexGood[indexGood == i] ) != 0): continue
    
    label = open('../images/1'+str(i).zfill(3)+'-receipt.txt', 'r') 
    labelText = label.read()
    
    imgBlurLevel = blurLevel(img)
    noiseLevel = estimate_noise(img)
    imgAngle = getAngle(img)
    
    text =''
    text = pytesseract.image_to_string(img)
    accuracy = accuracyByLevenshteinDistance(text,labelText)
    
    #descartadas naive
    if((noiseLevel >= 19 and imgBlurLevel >= 5253) or imgBlurLevel <= 302.16):
        if(imgAngle != None):
            if(imgAngle > 7):
                indexBad = np.append(indexBad,i)
                accuracyBad = np.append(accuracyBad, accuracy)
            else:
                indexMedium = np.append(indexMedium,i)
                accuracyMedium = np.append(accuracyMedium, accuracy)
                
        else:
            indexBad = np.append(indexBad,i)
            accuracyBad = np.append(accuracyBad, accuracy)  
    else:
        indexMedium = np.append(indexMedium,i)
        accuracyMedium = np.append(accuracyMedium, accuracy)
        
    
        
    
        
    
        

In [30]:
print(np.mean(accuracyAll))
print(np.mean(accuracyGood))
print(np.mean(accuracyMedium))
print(np.mean(accuracyBad))

0.2568223104924695
0.4515166453043116
0.30401889757080014
0.13447472834198768


In [31]:

print(np.mean(accuracyAllOriginal))

0.5981931123719203
